In [1]:
import pandas as pd
import warnings, pickle, os, json
warnings.filterwarnings("ignore")

gap = 40
min_change = 20

In [2]:
input_ = {
    "work_id": "FC236606T1-2",
    "op": 1,
    "model_id": "20230807120000000",
    "speed": "1000",
    "l_angle_ori": "334",
    "l_weight_ori": "167",
    "f_angle_ori": "197",
    "f_weight_ori": "81.3"
}

In [3]:
work_id = input_["work_id"]
op = input_["op"]
model_id = input_["model_id"]
speed = int(input_["speed"])
l_angle_ori  = int(input_["l_angle_ori"])
l_weight_ori = round(float(input_["l_weight_ori"]))
f_angle_ori  = int(input_["f_angle_ori"])
f_weight_ori = round(float(input_["f_weight_ori"]))
work_id, op, speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori

('FC236606T1-2', 1, 1000, 334, 167, 197, 81)

In [4]:
root = r"C:\Users\tzuli\Documents\python\teco"

# 取得predict位置
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
output_json = os.path.join(pred_path, "output.json")

# 取得model位置     
model_detail = os.path.join(root, "data", "train", model_id, "model")

### Load model

In [5]:
side = "L"
features = pickle.load(open(os.path.join(model_detail, f"feat_order.pkl"), "rb"))
l_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
l_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
l_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
l_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
l_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

side = "F"
f_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
f_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
f_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
f_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
f_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

### Generate candidate values

In [6]:
l_weight_change, f_weight_change = [], []
for i in range(gap+1):
    l_weight_change.extend([max(min_change, l_weight_ori + i), max(min_change, l_weight_ori - i)])
    f_weight_change.extend([max(min_change, f_weight_ori + i), max(min_change, f_weight_ori - i)])
l_weight_change = list(set(l_weight_change))
f_weight_change = list(set(f_weight_change))

print(f"l_weight_change = {l_weight_change}")
print(f"f_weight_change = {f_weight_change}")

l_weight_change = [127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207]
f_weight_change = [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121]


In [7]:
l_angle_change0, f_angle_change0 = [], []
for i in range(0, gap+1, 10):
    l_angle_change0.extend([l_angle_ori + i, l_angle_ori - i])
    f_angle_change0.extend([f_angle_ori + i, f_angle_ori - i])
l_angle_change0 = list(set(l_angle_change0))
f_angle_change0 = list(set(f_angle_change0))


l_angle_change = []
for angle in l_angle_change0:
    if angle > 360:
        l_angle_change.append(angle - 360)
    elif angle < 0:
        l_angle_change.append(360 - angle)
    else:
        l_angle_change.append(angle)


f_angle_change = []
for angle in f_angle_change0:
    if angle > 360:
        f_angle_change.append(angle - 360)
    elif angle < 0:
        f_angle_change.append(360 - angle)
    else:
        f_angle_change.append(angle)
        
print(f"l_angle_change = {l_angle_change}")
print(f"f_angle_change = {f_angle_change}")

l_angle_change = [354, 324, 294, 4, 334, 304, 14, 344, 314]
f_angle_change = [227, 197, 167, 237, 207, 177, 217, 187, 157]


### Generate candidate combinations

In [8]:
# 循環生成所有组合
all_combinations = []
for a in l_angle_change :
    for b in l_weight_change:
        for c in f_angle_change:
            for d in f_weight_change:
                x = [speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori, a, b, c, d]
                all_combinations.append(x)
                
len(all_combinations)

531441

In [3]:
import pandas as pd
a = pd.array([1,2,3])
a

<IntegerArray>
[1, 2, 3]
Length: 3, dtype: Int64

In [ ]:
# 

In [9]:
df_comb = pd.DataFrame(
    all_combinations, 
    columns = ['平衡轉速', '初始_L側角度', '初始_L側不平衡量', '初始_F側角度', '初始_F側不平衡量', '平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']
    )
df_pred = df_comb.copy()
df_pred.head(3)

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重
0,1000,334,167,197,81,354,127,227,41
1,1000,334,167,197,81,354,127,227,42
2,1000,334,167,197,81,354,127,227,43


### Generate features

In [10]:
df_pred["預期_F側不平衡量"] = df_pred["初始_F側不平衡量"] - df_pred["平衡_F側配重"]
df_pred["預期_L側不平衡量"] = df_pred["初始_L側不平衡量"] - df_pred["平衡_L側配重"]

df_pred["初始_不平衡量_diff"] = abs(df_pred["初始_L側不平衡量"] - df_pred["初始_F側不平衡量"])
df_pred["平衡_不平衡量_diff"] = abs(df_pred["平衡_L側配重"] - df_pred["平衡_F側配重"])

diff = abs(df_pred["初始_L側角度"] - df_pred["初始_F側角度"])
df_pred["初始_角度_diff"] = diff.apply(lambda X: min(X, 360 - X))
diff = abs(df_pred["平衡_L側角度"] - df_pred["平衡_F側角度"])
df_pred["平衡_角度_diff"] = diff.apply(lambda X: min(X, 360 - X))
df_pred.head(3)

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,預期_F側不平衡量,預期_L側不平衡量,初始_不平衡量_diff,平衡_不平衡量_diff,初始_角度_diff,平衡_角度_diff
0,1000,334,167,197,81,354,127,227,41,40,40,86,86,137,127
1,1000,334,167,197,81,354,127,227,42,39,40,86,85,137,127
2,1000,334,167,197,81,354,127,227,43,38,40,86,84,137,127


In [11]:
l_df, f_df = df_pred.copy(), df_pred.copy()

### Feature engineering

In [12]:
for col in features[1:]:    
    max_ = l_outlier_boundary[col]["max"]
    max_index = l_df.index[l_df[col] > max_].to_list()
    l_df.loc[max_index, col] = max_

    min_ = l_outlier_boundary[col]["min"]
    min_index = l_df.index[l_df[col] < min_].to_list()
    l_df.loc[min_index, col] = min_

    max_ = f_outlier_boundary[col]["max"]
    max_index = f_df.index[f_df[col] > max_].to_list()
    f_df.loc[max_index, col] = max_

    min_ = f_outlier_boundary[col]["min"]
    min_index = f_df.index[f_df[col] < min_].to_list()
    f_df.loc[min_index, col] = min_

In [13]:
l_df[l_skew_feat] = l_pt.transform(l_df[l_skew_feat])
f_df[f_skew_feat] = f_pt.transform(f_df[f_skew_feat])
l_df[features]    = l_scaler.transform(l_df[features])
f_df[features]    = f_scaler.transform(f_df[features])

### Predict

In [14]:
pred_l = l_model.predict(l_df)
pred_f = f_model.predict(f_df)

In [15]:
df_comb["最終_L側不平衡量"] = pred_l
df_comb["最終_F側不平衡量"] = pred_f

### Find best combination

In [16]:
df_comb["總不平衡量"] = df_comb.eval("(abs(最終_L側不平衡量) + abs(最終_F側不平衡量))")
# 先嚴格篩選，在寬鬆篩選
df_comb1 = df_comb.query("((平衡_L側配重 > 0) and (平衡_F側配重 > 0)) and ((最終_L側不平衡量 > 0) and (最終_F側不平衡量 > 0))")
if len(df_comb1) == 0:
    df_comb1 = df_comb.query("(最終_L側不平衡量 > 0) and (最終_F側不平衡量 > 0)")
if len(df_comb1) == 0:
    df_comb1 = df_comb.copy()
comb_index = df_comb1["總不平衡量"].idxmin()
result = df_comb.loc[[comb_index]]
result

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,最終_L側不平衡量,最終_F側不平衡量,總不平衡量
211699,1000,334,167,197,81,4,174,237,87,8.204687,8.586943,16.79163


In [17]:
result = result[['平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']]
result = result.astype(str)
result.columns = ["l_angle_pred", "l_weight_pred", "f_angle_pred", "f_weight_pred"]
result = result.to_dict(orient = "records")[0]

### Save output

In [18]:
result = {
    "status":   "success",
    "reason":   "",
    "work_id":  work_id,
    "op":       op,
    "model_id": model_id,
    "predict":  result
}
result

{'status': 'success',
 'reason': '',
 'work_id': 'FC236606T1-2',
 'op': 1,
 'model_id': '20230807120000000',
 'predict': {'l_angle_pred': '4',
  'l_weight_pred': '174',
  'f_angle_pred': '237',
  'f_weight_pred': '87'}}

In [19]:
with open(output_json, 'w') as file:
    json.dump(result, file, indent = 4)